In [1]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE=88584
MAX_LEN=250
BATCH=64
(train_data, train_labels),(test_data, test_labels)=imdb.load_data(num_words=VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 0s 0us/step


In [2]:
# Making the review inputs of same length to pass into the neural Network
#from tensorflow.keras.preprocessing.sequence import pad_sequences

train_data=tf.keras.utils.pad_sequences(train_data, MAX_LEN)
test_data=tf.keras.utils.pad_sequences(test_data, MAX_LEN)

In [3]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop",metrics=['acc'])
history=model.fit(train_data, train_labels,epochs=10,validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 57s 80ms/step - loss: 0.4517 - acc: 0.7810 - val_loss: 0.3234 - val_acc: 0.8668
Epoch 2/10
625/625 [==============================] - 23s 37ms/step - loss: 0.2583 - acc: 0.8988 - val_loss: 0.2852 - val_acc: 0.8858
Epoch 3/10
625/625 [==============================] - 14s 23ms/step - loss: 0.2019 - acc: 0.9262 - val_loss: 0.2900 - val_acc: 0.8790
Epoch 4/10
625/625 [==============================] - 11s 18ms/step - loss: 0.1643 - acc: 0.9421 - val_loss: 0.2923 - val_acc: 0.8810
Epoch 5/10
625/625 [==============================] - 9s 15ms/step - loss: 0.1359 - acc: 0.9537 - val_loss: 0.3041 - val_acc: 0.8790
Epoch 6/10
625/625 [==============================] - 9s 14ms/step - loss: 0.1177 - acc: 0.9614 - val_loss: 0.3147 - val_acc: 0.8722
Epoch 7/10
625/625 [==============================] - 9s 14ms/step - loss: 0.0978 - acc: 0.9675 - val_loss: 0.3568 - val_acc: 0.8748
Epoch 8/10
625/625 [==============================] - 8s 13ms/ste

In [7]:
results=model.evaluate(test_data,test_labels)
print(results)

782/782 [==============================] - 4s 6ms/step - loss: 0.5355 - acc: 0.8587
[0.5355368852615356, 0.8587200045585632]


In [13]:
word_index= imdb.get_word_index()

def encode_text(text):
  tokens=keras.preprocessing.text.text_to_word_sequence(text)
  tokens=[word_index[word] if word in word_index else 0 for word in tokens]
  return tf.keras.utils.pad_sequences([tokens],MAX_LEN)[0]

text="the movie was Amazing! I loved it"
encoded=encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   1  17  13 47

In [14]:
reverse_word_index={value: key for (key, value) in word_index.items()}

def decode_text(integers):
  PAD=0
  text=""
  for num in integers:
    if num!=PAD:
      text+=reverse_word_index[num] + " "
  return text[:-1]

print(decode_text(encoded))

the movie was amazing i loved it


In [21]:
def predict_text(text):
  encoded_text=encode_text(text)
  pred=np.zeros((1,250))
  pred[0]=encoded_text
  result=model.predict(pred)
  return result[0]

def positiveOrNegative(result):
  if result<0.5:
    print("negative")
  else:
    print("positive")

#positive
pos="The movie was amazing, I loved it!"
print("The review is: ")
positiveOrNegative(predict_text(pos))

#negative
neg="cringe. I will never watch it again"
print("The review is: ")
positiveOrNegative(predict_text(neg))

The review is: 
1/1 [==============================] - 0s 26ms/step
positive
The review is: 
1/1 [==============================] - 0s 26ms/step
negative
